<a href="https://colab.research.google.com/github/davehusen/scripts/blob/master/%5BTEMPLATE%5D_Waste_Characterization_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up things with git and code
First step we need to clone and get everything in order

In [1]:
import os
import subprocess
import shutil
from google.colab import drive


In [2]:
drive = drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
repository_url = "git@github.com:orange-industrial/pipeline.git"  # Replace with your repository URL
branch = "dave/oi-177-add-ci-reorg-pipeline-code" # Replace with your branch name
ssh_key_path = "/content/drive/MyDrive/Orange Industrial/Engineering/Colab Deploy Keys/" # Set the SSH key path
ssh_key_name = 'id_ed25519_pipeline_deploy' # Github read-only deploy key for repo at repository_url
target_directory = "/content/drive/MyDrive/HopperRun"  # Set the target directory path in Google Drive, give full path!
pipeline = "waste-characterization"

In [4]:
# Copy deploy key to instance for authentication to github repo
! mkdir -p /root/.ssh
shutil.copy2(ssh_key_path+ssh_key_name, "/root/.ssh/id_ed25519")
! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
! chmod go-rwx /root/.ssh/
! ssh -T git@github.com

# github.com:22 SSH-2.0-babeld-d815c248
Hi orange-industrial/pipeline! You've successfully authenticated, but GitHub does not provide shell access.


In [5]:
# Save clone_or_pull.sh to instance
sh = """
#!/bin/bash
set -x

REPO_SRC="$1"
LOCAL_REPO="$2"

LOCAL_REPO_VC_DIR="$LOCAL_REPO/.git"

if [ ! -d "$LOCAL_REPO_VC_DIR" ]
then
    git clone "$REPO_SRC" "$LOCAL_REPO"
else
    cd "$LOCAL_REPO"
    git pull "$REPO_SRC"
fi
"""

with open('clone_or_pull.sh', 'w') as file:
  file.write(sh)

# Cloning the repo and swithcing to branch

In [6]:
! bash clone_or_pull.sh git@github.com:orange-industrial/pipeline pipeline

+ REPO_SRC=git@github.com:orange-industrial/pipeline
+ LOCAL_REPO=pipeline
+ LOCAL_REPO_VC_DIR=pipeline/.git
+ '[' '!' -d pipeline/.git ']'
+ cd pipeline
+ git pull git@github.com:orange-industrial/pipeline
From github.com:orange-industrial/pipeline
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [7]:
! cd pipeline/ ; git checkout "$branch"

Already on 'dave/oi-177-add-ci-reorg-pipeline-code'
Your branch is up to date with 'origin/dave/oi-177-add-ci-reorg-pipeline-code'.


# Running code

In [8]:
os.chdir("/content/pipeline/pipelines/" + pipeline)

In [9]:
!pip install wandb -qqq
!pip install ultralytics --quiet
!pip install roboflow

In [20]:
! git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (6/6), 508 bytes | 508.00 KiB/s, done.
From github.com:orange-industrial/pipeline
   985f5cd..90777fb  dave/oi-177-add-ci-reorg-pipeline-code -> origin/dave/oi-177-add-ci-reorg-pipeline-code
Updating 985f5cd..90777fb
Fast-forward
 pipelines/waste-characterization/config/roboflow_config.json | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [21]:
# WANDB API KEY: ed9e154fbfc5ef1458bc77b312183e4cbd097648
!python main.py --rf_api_key DooHVuDzvvLAevjx9YIt \
--sweep_flag True \
--sweep_json config/sweep_config.json

Dataset path: /content/drive/MyDrive/Orange Industrial/Engineering/Datasets/waste-characterization-3
Device we are running on: cuda
Create sweep with ID: gmirhivp
Sweep URL: https://wandb.ai/orange-industrial/waste-characterization/sweeps/gmirhivp
wandb: Agent Starting Run: 4at0mcd2 with config:
wandb: 	backbone: yolov8
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.0005
wandb: 	momentum: 0.9
wandb: 	num_epochs: 100
wandb: 	num_workers: 8
wandb: 	weight_decay: 0.004
wandb: Currently logged in as: davehusen (orange-industrial). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/drive/MyDrive/Orange Industrial/Engineering/Models/waste-characterization-3/wandb/run-20230803_212244-4at0mcd2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run light-sweep-1
wandb: ⭐️ View project at https://wandb.ai/orange-industrial/waste-c

# 2023-08-02: NOT UPDATED BEYOND THIS POINT

In [ ]:
import os

This is where all of the metrics and **in sample** results are in

In [ ]:
os.listdir("/content/drive/MyDrive/yolo_data/runs/detect/train")

['weights',
 'args.yaml',
 'events.out.tfevents.1690180156.a865732f13ce.4874.0',
 'labels_correlogram.jpg',
 'labels.jpg',
 'train_batch0.jpg',
 'train_batch1.jpg',
 'train_batch2.jpg',
 'val_batch0_pred.jpg',
 'val_batch0_labels.jpg',
 'val_batch1_pred.jpg',
 'val_batch1_labels.jpg',
 'val_batch2_pred.jpg',
 'val_batch2_labels.jpg',
 'PR_curve.png',
 'F1_curve.png',
 'P_curve.png',
 'R_curve.png',
 'confusion_matrix_normalized.png',
 'confusion_matrix.png',
 'results.csv',
 'results.png']

In [ ]:
path2_test_data = "/content/drive/MyDrive/yolo_data/Brentwood---Recycling---Plastic-Film-1/test"
list_of_test_files = os.listdir(path2_test_data+"/images")

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('/content/drive/MyDrive/Orange Industrial/Engineering/Models/hopper_states-devout-bush-5-backbone_yolov8-epochs_1-batch_32-workers_2-lr_0.1-20230724-063004.pth')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


In [ ]:
res = model(list_of_test_files[0])
res_plotted = res[0].plot()
cv2.imshow("result", res_plotted)

AssertionError: ignored